In [35]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import seaborn as sns
import time
import matplotlib.pyplot as plt
%matplotlib inline
# Any results you write to the current directory are saved as output.

from sklearn.preprocessing import MinMaxScaler, StandardScaler


['sample_submission.csv', 'test.csv', 'train.csv']


In [36]:
def logloss(y,yp):
    yp = np.clip(yp,1e-5,1-1e-5)
    return -y*np.log(yp)-(1-y)*np.log(1-yp)
    
def reverse(tr,te):
    reverse_list = [0,1,2,3,4,5,6,7,8,11,15,16,18,19,
                22,24,25,26,27,41,29,
                32,35,37,40,48,49,47,
                55,51,52,53,60,61,62,103,65,66,67,69,
                70,71,74,78,79,
                82,84,89,90,91,94,95,96,97,99,
                105,106,110,111,112,118,119,125,128,
                130,133,134,135,137,138,
                140,144,145,147,151,155,157,159,
                161,162,163,164,167,168,
                170,171,173,175,176,179,
                180,181,184,185,187,189,
                190,191,195,196,199]
    reverse_list = ['var_%d'%i for i in reverse_list]
    for col in reverse_list:
        tr[col] = tr[col]*(-1)
        te[col] = te[col]*(-1)
    return tr,te

def reverse(tr):
    reverse_list = [0,1,2,3,4,5,6,7,8,11,15,16,18,19,
                22,24,25,26,27,41,29,
                32,35,37,40,48,49,47,
                55,51,52,53,60,61,62,103,65,66,67,69,
                70,71,74,78,79,
                82,84,89,90,91,94,95,96,97,99,
                105,106,110,111,112,118,119,125,128,
                130,133,134,135,137,138,
                140,144,145,147,151,155,157,159,
                161,162,163,164,167,168,
                170,171,173,175,176,179,
                180,181,184,185,187,189,
                190,191,195,196,199]
    reverse_list = ['var_%d'%i for i in reverse_list]
    for col in reverse_list:
        tr[col] = tr[col]*(-1)
    return tr

def scale(tr,te):
    for col in tr.columns:
        if col.startswith('var_'):
            mean,std = tr[col].mean(),tr[col].std()
            tr[col] = (tr[col]-mean)/std
            te[col] = (te[col]-mean)/std
    return tr,te

def scale(tr):
    for col in tr.columns:
        if col.startswith('var_'):
            mean,std = tr[col].mean(),tr[col].std()
            tr[col] = (tr[col]-mean)/std
    return tr

def getp_vec_sum(x,x_sort,y,std,c=0.5):
    # x is sorted
    left = x - std/c
    right = x + std/c
    p_left = np.searchsorted(x_sort,left)
    p_right = np.searchsorted(x_sort,right)
    p_right[p_right>=y.shape[0]] = y.shape[0]-1
    p_left[p_left>=y.shape[0]] = y.shape[0]-1
    return (y[p_right]-y[p_left])

def get_pdf(tr,col,x_query=None,smooth=3):
    std = tr[col].std()
    df = tr.groupby(col).agg({'target':['sum','count']})
    cols = ['sum_y','count_y']
    df.columns = cols
    df = df.reset_index()
    df = df.sort_values(col)
    y,c = cols
    
    df[y] = df[y].cumsum()
    df[c] = df[c].cumsum()
    
    if x_query is None:
        rmin,rmax,res = -5.0, 5.0, 501
        x_query = np.linspace(rmin,rmax,res)
    
    dg = pd.DataFrame()
    tm = getp_vec_sum(x_query,df[col].values,df[y].values,std,c=smooth)
    cm = getp_vec_sum(x_query,df[col].values,df[c].values,std,c=smooth)+1
    dg['res'] = tm/cm
    dg.loc[cm<500,'res'] = 0.1
    return dg['res'].values

def get_pdfs(tr):
    y = []
    for i in range(200):
        name = 'var_%d'%i
        res = get_pdf(tr,name)
        y.append(res)
    return np.vstack(y)

def print_corr(corr_mat,col,bar=0.95):
    #print(col)
    cols = corr_mat.loc[corr_mat[col]>bar,col].index.values  
    cols_ = ['var_%s'%(i.split('_')[-1]) for i in cols]
    print(col,"#####", cols)
    return cols

In [37]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv('../input/test.csv')

In [38]:
# train_df,test_df = reverse(train_df,test_df)
# train_df,test_df = scale(train_df,test_df)

In [39]:
# %%time
# pdfs = get_pdfs(train_df)

In [40]:
# %%time
# df_pdf = pd.DataFrame(pdfs.T,columns=['var_prob_%d'%i for i in range(200)])
# corr_mat = df_pdf.corr(method='pearson')

In [41]:
# corr_mat.head()
# distance_mat =  (1 - corr_mat)
# distance_mat.head()

In [42]:
special_features =[ 'var_17',
'var_100',
'var_124']

In [43]:
%%time
from sklearn.cluster import DBSCAN
def get_group_features(df):
    tr = df.copy()
    tr = reverse(tr)
    tr = scale(tr)
    pdfs = get_pdfs(tr)
    df_pdf = pd.DataFrame(pdfs.T,columns=['var_prob_%d'%i for i in range(200)])
    corr_mat = df_pdf.corr(method='pearson')
    distance_mat =  (1 - corr_mat)
    distance_mat.head()

    groups =[]
    skip_list = []
    for i in range(0,200):
        if(i not in skip_list):
            cols = print_corr(corr_mat,'var_prob_'+str(i), bar=0.95)
            if(len(cols)>0):
                groups.append(cols)
                for e,v in enumerate(cols):
                    skip_list.append(int(v[9:]))
    print(len(groups))
    
    return groups

def get_cluster_centre(groups):
    centres = []
    for i, features in enumerate(groups):
        centres.append(features[0].replace('prob_', ''))
    return centres

groups = get_group_features(train_df)
for i,v in enumerate(groups):
    print(i,"########", v)
centres = get_cluster_centre(groups)

var_prob_0 ##### ['var_prob_0' 'var_prob_2' 'var_prob_6' 'var_prob_26' 'var_prob_44'
 'var_prob_76' 'var_prob_80' 'var_prob_86' 'var_prob_106' 'var_prob_110'
 'var_prob_111' 'var_prob_123' 'var_prob_155' 'var_prob_157'
 'var_prob_162' 'var_prob_174' 'var_prob_181' 'var_prob_191'
 'var_prob_198']
var_prob_1 ##### ['var_prob_1' 'var_prob_5' 'var_prob_15' 'var_prob_24' 'var_prob_34'
 'var_prob_40' 'var_prob_43' 'var_prob_53' 'var_prob_78' 'var_prob_94'
 'var_prob_109' 'var_prob_116' 'var_prob_122' 'var_prob_127'
 'var_prob_137' 'var_prob_145' 'var_prob_165' 'var_prob_166'
 'var_prob_171' 'var_prob_180']
var_prob_3 ##### ['var_prob_3' 'var_prob_154']
var_prob_4 ##### ['var_prob_4' 'var_prob_35' 'var_prob_173' 'var_prob_188']
var_prob_7 ##### ['var_prob_7']
var_prob_8 ##### ['var_prob_8']
var_prob_9 ##### ['var_prob_9' 'var_prob_12' 'var_prob_49' 'var_prob_55' 'var_prob_75'
 'var_prob_79' 'var_prob_83' 'var_prob_94' 'var_prob_95' 'var_prob_99'
 'var_prob_113' 'var_prob_114' 'var_prob_122' '

In [44]:
centres

['var_0',
 'var_1',
 'var_3',
 'var_4',
 'var_7',
 'var_8',
 'var_9',
 'var_10',
 'var_11',
 'var_6',
 'var_14',
 'var_16',
 'var_17',
 'var_19',
 'var_20',
 'var_21',
 'var_2',
 'var_25',
 'var_27',
 'var_29',
 'var_30',
 'var_31',
 'var_32',
 'var_37',
 'var_38',
 'var_39',
 'var_41',
 'var_42',
 'var_45',
 'var_46',
 'var_47',
 'var_23',
 'var_50',
 'var_23',
 'var_54',
 'var_57',
 'var_58',
 'var_59',
 'var_60',
 'var_61',
 'var_62',
 'var_56',
 'var_64',
 'var_65',
 'var_6',
 'var_12',
 'var_69',
 'var_23',
 'var_12',
 'var_72',
 'var_73',
 'var_74',
 'var_77',
 'var_84',
 'var_87',
 'var_93',
 'var_96',
 'var_98',
 'var_100',
 'var_101',
 'var_103',
 'var_104',
 'var_44',
 'var_112',
 'var_117',
 'var_120',
 'var_124',
 'var_126',
 'var_129',
 'var_136',
 'var_6',
 'var_140',
 'var_142',
 'var_99',
 'var_144',
 'var_150',
 'var_152',
 'var_153',
 'var_158',
 'var_159',
 'var_160',
 'var_161',
 'var_128',
 'var_2',
 'var_176',
 'var_182',
 'var_183',
 'var_185',
 'var_186',
 'var_

In [45]:
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,...,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,2.9252,3.1821,14.0137,0.5745,8.7989,14.5691,5.7487,-7.2393,4.2840,30.7133,10.5350,16.2191,2.5791,2.4716,14.3831,13.4325,-5.1488,-0.4073,4.9306,5.9965,-0.3085,12.9041,-3.8766,16.8911,11.1920,10.5785,0.6764,7.8871,...,15.4576,5.3133,3.6159,5.0384,6.6760,12.6644,2.7004,-0.6975,9.5981,5.4879,-4.7645,-8.4254,20.8773,3.1531,18.5618,7.7423,-10.1245,13.7241,-3.5189,1.7202,-8.4051,9.0164,3.0657,14.3691,25.8398,5.8764,11.8411,-19.7159,17.5743,0.5857,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,-0.4032,8.0585,14.0239,8.4135,5.4345,13.7003,13.8275,-15.5849,7.8000,28.5708,3.4287,2.7407,8.5524,3.3716,6.9779,13.8910,-11.7684,-2.5586,5.0464,0.5481,-9.2987,7.8755,1.2859,19.3710,11.3702,0.7399,2.7995,5.8434,...,29.4846,5.8683,3.8208,15.8348,-5.0121,15.1345,3.2003,9.3192,3.8821,5.7999,5.5378,5.0988,22.0330,5.5134,30.2645,10.4968,-7.2352,16.5721,-7.3477,11.0752,-5.5937,9.4878,-14.9100,9.4245,22.5441,-4.8622,7.6543,-15.9319,13.3175,-0.3566,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,-0.3249,-11.2648,14.1929,7.3124,7.5244,14.6472,7.6782,-1.7395,4.7011,20.4775,17.7559,18.1377,1.2145,3.5137,5.6777,13.2177,-7.9940,-2.9029,5.8463,6.1439,-11.1025,12.4858,-2.2871,19.0422,11.0449,4.1087,4.6974,6.9346,...,13.2070,5.8442,4.7086,5.7141,-1.0410,20.5092,3.2790,-5.5952,7.3176,5.7690,-7.0927,-3.9116,7.2569,-5.8234,25.6820,10.9202,-0.3104,8.8438,-9.7009,2.4013,-4.2935,9.3908,-13.2648,3.1545,23.0866,-5.3000,5.3745,-6.2660,10.1934,-0.8417,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,2.3061,2.8102,13.8463,11.9704,6.4569,14.8372,10.7430,-0.4299,15.9426,13.7257,20.3010,12.5579,6.8202,2.7229,12.1354,13.7367,0.8135,-0.9059,5.9070,2.8407,-15.2398,10.4407,-2.5731,6.1796,10.6093,-5.9158,8.1723,2.8521,...,31.8833,5.9684,7.2084,3.8899,-11.0882,17.2502,2.5881,-2.7018,0.5641,5.3430,-7.1541,-6.1920,18.2366,11.7134,14.7483,8.1013,11.8771,13.9552,-10.4701,5.6961,-3.7546,8.4117,1.8986,7.2601,-0.4639,-0.0498,7.9336,-12.8279,12.4124,1.8489,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,-9.4458,-12.1419,13.8481,7.8895,7.7894,15.0553,8.4871,-3.0680,6.5263,11.3152,21.4246,18.9608,10.1102,2.7142,14.2080,13.5433,3.1736,-3.3423,5.9015,7.9352,-3.1582,9.4668,-0.0083,19.3239,12.4057,0.6329,2.7922,5.8184,...,33.5107,5.6953,5.4663,18.2201,6.5769,21.2607,3.2304,-1.7759,3.1283,5.5518,1.4493,-2.6627,19.8056,2.3705,18.4685,16.3309,-3.3456,13.5261,1.7189,5.1743,-7.6938,9.7685,4.8910,12.2198,11.8503,-7.8931,6.4209,5.9270,16.0201,-0.2829,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [46]:
features = [c for c in train_df.columns if c not in ['ID_code', 'target']]

n_bins = 200
def fe_count_bins(df):
    temp = df.copy()
    scaler = MinMaxScaler()
    for col in centres:
        temp[col] = pd.cut(temp[col], n_bins, labels=False)
        df[col+'_bin_counts'] = temp[col].map(temp.groupby(col)['ID_code'].count().to_dict())
        df[col+'_bin_counts'] = scaler.fit_transform(np.array(df[col+'_bin_counts']).reshape(-1, 1))
# for df in [train_df, test_df]:
#     for col in special_features:
#         df[col] = pd.cut(df[col], n_bins, labels=False)
    return df

train_df = fe_count_bins(train_df)
test_df = fe_count_bins(test_df)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.

In [47]:
# scaler = MinMaxScaler()
# col='var_17'
# temp = train_df.copy()
# temp[col] = pd.cut(temp[col], n_bins, labels=False)
# train_df[col+'_count_bins'] = temp[col].map(temp.groupby(col)['ID_code'].count().to_dict())
# train_df[col+'_count_bins'].plot.kde()
# train_df[col+'_count_bins'] = scaler.fit_transform(np.array(train_df[col+'_count_bins']).reshape(-1, 1))
# train_df[col+'_count_bins'].plot.kde()
# plt.show()

In [48]:
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,...,var_84_bin_counts,var_87_bin_counts,var_93_bin_counts,var_96_bin_counts,var_98_bin_counts,var_100_bin_counts,var_101_bin_counts,var_103_bin_counts,var_104_bin_counts,var_44_bin_counts,var_112_bin_counts,var_117_bin_counts,var_120_bin_counts,var_124_bin_counts,var_126_bin_counts,var_129_bin_counts,var_136_bin_counts,var_140_bin_counts,var_142_bin_counts,var_99_bin_counts,var_144_bin_counts,var_150_bin_counts,var_152_bin_counts,var_153_bin_counts,var_158_bin_counts,var_159_bin_counts,var_160_bin_counts,var_161_bin_counts,var_128_bin_counts,var_176_bin_counts,var_182_bin_counts,var_183_bin_counts,var_185_bin_counts,var_186_bin_counts,var_82_bin_counts,var_189_bin_counts,var_192_bin_counts,var_193_bin_counts,var_85_bin_counts,var_199_bin_counts
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,2.9252,3.1821,14.0137,0.5745,8.7989,14.5691,5.7487,-7.2393,4.2840,30.7133,10.5350,16.2191,2.5791,2.4716,14.3831,13.4325,-5.1488,-0.4073,4.9306,5.9965,-0.3085,12.9041,-3.8766,16.8911,11.1920,10.5785,0.6764,7.8871,...,0.875352,0.698146,0.571476,0.868097,0.921519,0.288906,0.958815,0.911614,0.318569,0.968035,0.950470,0.951878,0.806717,1.000000,0.422430,1.000000,0.941204,0.695994,0.537856,0.418442,0.981103,0.779805,0.922498,0.903094,0.928329,0.931874,0.937912,0.456938,0.958688,0.686713,0.972093,0.573833,0.139039,0.643271,0.969939,1.000000,0.735456,0.900449,0.860816,0.896362
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,-0.4032,8.0585,14.0239,8.4135,5.4345,13.7003,13.8275,-15.5849,7.8000,28.5708,3.4287,2.7407,8.5524,3.3716,6.9779,13.8910,-11.7684,-2.5586,5.0464,0.5481,-9.2987,7.8755,1.2859,19.3710,11.3702,0.7399,2.7995,5.8434,...,0.895456,0.956978,0.833278,0.635783,0.960217,0.806513,0.858363,0.813834,0.198296,0.334378,0.686166,0.865111,0.866690,0.346907,0.168917,0.896114,0.685325,1.000000,0.101949,0.755909,0.973004,0.459482,0.889391,0.903094,0.798134,1.000000,0.893697,0.651116,0.951022,0.792361,0.293953,0.892301,0.939107,0.915003,0.952560,0.673222,0.860073,0.148980,0.730940,1.000000
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,-0.3249,-11.2648,14.1929,7.3124,7.5244,14.6472,7.6782,-1.7395,4.7011,20.4775,17.7559,18.1377,1.2145,3.5137,5.6777,13.2177,-7.9940,-2.9029,5.8463,6.1439,-11.1025,12.4858,-2.2871,19.0422,11.0449,4.1087,4.6974,6.9346,...,0.556494,0.929346,0.913626,0.898220,0.968897,0.904203,0.932195,0.771563,0.224872,0.512065,0.964560,0.702515,0.962988,0.912563,0.357950,0.878176,0.988241,0.847997,0.978261,0.938009,0.916313,0.817565,0.565463,0.530945,0.141645,0.934345,0.877705,0.690191,0.952726,0.971150,0.423721,0.570888,0.925237,0.538957,0.187882,0.368201,0.991733,0.900449,0.938830,0.927242
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,2.3061,2.8102,13.8463,11.9704,6.4569,14.8372,10.7430,-0.4299,15.9426,13.7257,20.3010,12.5579,6.8202,2.7229,12.1354,13.7367,0.8135,-0.9059,5.9070,2.8407,-15.2398,10.4407,-2.5731,6.1796,10.6093,-5.9158,8.1723,2.8521,...,0.872135,0.744316,0.984265,0.963031,0.356239,0.429761,0.246610,0.690436,0.612947,0.870573,0.697267,0.339774,0.870802,0.791035,0.537614,0.949178,0.961900,0.943848,0.972639,0.311120,0.418434,0.903691,0.986080,0.733713,0.906701,0.850335,0.883819,0.330941,0.461244,0.254774,0.999535,0.973075,0.724628,0.936574,1.000000,0.691213,0.758726,0.905980,0.928635,0.795262
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,-9.4458,-12.1419,13.8481,7.8895,7.7894,15.0553,8.4871,-3.0680,6.5263,11.3152,21.4246,18.9608,10.1102,2.7142,14.2080,13.5433,3.1736,-3.3423,5.9015,7.9352,-3.1582,9.4668,-0.0083,19.3239,12.4057,0.6329,2.7922,5.8184,...,0.289505,0.36

In [49]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier,Pool
from IPython.display import display
import matplotlib.patches as patch
import matplotlib.pyplot as plt
from sklearn.svm import NuSVR
from scipy.stats import norm
from sklearn import svm
import lightgbm as lgb
import xgboost as xgb
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import time
import glob
import sys
import os
import gc
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import scipy
import pickle
from sklearn.mixture import GaussianMixture

In [50]:
features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
target = train_df['target']

In [51]:
param = {
    'bagging_freq': 5,
    'bagging_fraction': 0.4,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.0083,
    'learning_rate': 0.01,
    'max_depth': -1,  
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'feature',
    'objective': 'binary', 
    'verbosity': 1
}

In [ ]:
folds = StratifiedKFold(n_splits=10, shuffle=False, random_state=44000)
oof = np.zeros(len(train_df))
predictions = np.zeros(len(test_df))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx])

    num_round = 1000000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 3000)
    oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test_df[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))

Fold 0
Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.90694	valid_1's auc: 0.877801
[2000]	training's auc: 0.918032	valid_1's auc: 0.887387
[3000]	training's auc: 0.923658	valid_1's auc: 0.89117
[4000]	training's auc: 0.928487	valid_1's auc: 0.89394
[5000]	training's auc: 0.931261	valid_1's auc: 0.894999
[6000]	training's auc: 0.934137	valid_1's auc: 0.89673
[7000]	training's auc: 0.9364	valid_1's auc: 0.897027
[8000]	training's auc: 0.938805	valid_1's auc: 0.897929
[9000]	training's auc: 0.940857	valid_1's auc: 0.898195
[10000]	training's auc: 0.942752	valid_1's auc: 0.898442
[11000]	training's auc: 0.944683	valid_1's auc: 0.898826
[12000]	training's auc: 0.946506	valid_1's auc: 0.898841
[13000]	training's auc: 0.948253	valid_1's auc: 0.899077
[14000]	training's auc: 0.949931	valid_1's auc: 0.899295
[15000]	training's auc: 0.951601	valid_1's auc: 0.899207
[16000]	training's auc: 0.95315	valid_1's auc: 0.899212
[17000]	training's auc: 0.954706	

In [ ]:
cols = (feature_importance_df[["Feature", "importance"]]
        .groupby("Feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:250].index)
best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

plt.figure(figsize=(14,28))
sns.barplot(x="importance", y="Feature", data=best_features.sort_values(by="importance",ascending=False))
plt.title('Features importance (averaged/folds)')
plt.tight_layout()
plt.savefig('FI.png')